# 输出解析器

语言模型输出文本。但是很多时候，您可能想获得比文本更结构化的信息。这就是输出解析器的作用。

输出解析器是帮助结构化语言模型响应的类。一个输出解析器必须实现两种主要方法：

- `get_format_instructions() -> str`：返回一个包含指示语言模型输出格式的字符串的方法。
- `parse(str) -> Any`：一个将字符串（假定为语言模型的响应）作为输入，并将其解析为某种结构的方法。

然后还有一种可选方法：

- `parse_with_prompt(str, PromptValue) -> Any`：一个方法，它将字符串（假定为语言模型的响应）和一个提示（假定为生成这种响应的提示）作为输入，并将其解析为某个结构。提示在很大程度上是提供的，以便OutputParser想要重试或以某种方式修复输出，需要来自提示的信息。


以下是主要类型的输出解析器“PydanticOutputParser”。请参见“examples”文件夹以了解其他选项。

In [1]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [2]:
model_name = 'text-davinci-003'
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [3]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

In [4]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [5]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [6]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)

In [7]:
output = model(_input.to_string())

In [8]:
parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')